# Constrained Optimization

###  Using `ipyopt` to solve a simple constrained optimization problem:
Author: "Eric Xu. Washington University"

#### Problem Definition:


The same model as Ipopt/examples/hs071. You can set Ipopt options by calling nlp.set.
For instance, to set the tolarance by calling: `nlp.set(tol=1e-8)`

For a complete list of Ipopt options, refer to http://www.coin-or.org/Ipopt/documentation/node59.html

\begin{equation*}
\begin{aligned}
& \underset{\mathbf{x}\ \epsilon\ \mathbb{R}^4}{\text{min}}
&& x_1x_4(x_1+x_2+x_3) \\
& \text{s.t.}
&& x_1x_2x_3x_4\geq25\\
&&& x_1^2 + x_2^2 + x_3^2 + x_4^2 = 40\\
&&& 1\leq \mathbf{x} \leq 5.

\end{aligned}
\end{equation*}

#### Import necessary libraries:

In [2]:
from __future__ import print_function
import ipyopt
from numpy import float_
import numpy as np

#### Define bounds on the variable and constraints:

In [3]:
nvar = 4 # number of variables
ncon = 2 # number of constraints

x_L = np.ones((nvar), dtype=float_) * 1.0 # lower bounds on x
x_U = np.ones((nvar), dtype=float_) * 5.0 # upper bounds on x

g_L = np.array([25.0, 40.0]) # lower bounds on g(x)
g_U = np.array([2.0e19, 40.0]) # upper bounds on g(x)

#### Define the objective function and its derivative:

In [4]:
# define f(x):
def eval_f(x):
    assert len(x) == nvar
    return x[0] * x[3] * (x[0] + x[1] + x[2]) + x[2]

# gradient of f(x):
def eval_grad_f(x, out):
    assert len(x) == nvar
    out[0] = x[0] * x[3] + x[3] * (x[0] + x[1] + x[2])
    out[1] = x[0] * x[3]
    out[2] = x[0] * x[3] + 1.0
    out[3] = x[0] * (x[0] + x[1] + x[2])
    return out

#### Define constraints and their derivatives:

In [5]:
# define the constraints g_1(x) and g_2(x):
def eval_g(x, out):
    assert len(x) == nvar
    out[0] = x[0] * x[1] * x[2] * x[3]
    out[1] = x[0] * x[0] + x[1] * x[1] + x[2] * x[2] + x[3] * x[3]
    return out

# gradient of constraints g_1(x) and g_2(x):
def eval_jac_g(x, out):
    assert len(x) == nvar
    out[()] = [x[1] * x[2] * x[3],
               x[0] * x[2] * x[3],
               x[0] * x[1] * x[3],
               x[0] * x[1] * x[2],
               2.0 * x[0],
               2.0 * x[1],
               2.0 * x[2],
               2.0 * x[3]]
    return out


eval_jac_g.sparsity_indices = (np.array([0, 0, 0, 0, 1, 1, 1, 1]),
                               np.array([0, 1, 2, 3, 0, 1, 2, 3]))

#### Define Lagrange:

In [8]:
# hessian of the lagrangian:
def eval_h(x, lagrange, obj_factor, out):
    out[0] = obj_factor * (2 * x[3])
    out[1] = obj_factor * (x[3])
    out[2] = 0
    out[3] = obj_factor * (x[3])
    out[4] = 0
    out[5] = 0
    out[6] = obj_factor * (2 * x[0] + x[1] + x[2])
    out[7] = obj_factor * (x[0])
    out[8] = obj_factor * (x[0])
    out[9] = 0
    out[1] += lagrange[0] * (x[2] * x[3])

    out[3] += lagrange[0] * (x[1] * x[3])
    out[4] += lagrange[0] * (x[0] * x[3])

    out[6] += lagrange[0] * (x[1] * x[2])
    out[7] += lagrange[0] * (x[0] * x[2])
    out[8] += lagrange[0] * (x[0] * x[1])
    out[0] += lagrange[1] * 2
    out[2] += lagrange[1] * 2
    out[5] += lagrange[1] * 2
    out[9] += lagrange[1] * 2
    return out


eval_h.sparsity_indices = (np.array([0, 1, 1, 2, 2, 2, 3, 3, 3, 3]),
                           np.array([0, 0, 1, 0, 1, 2, 0, 1, 2, 3]))

In [7]:
def apply_new(_x):
    return True

#### Set the problem up:

In [14]:
# feed in the problem data:
nlp = ipyopt.Problem(nvar, x_L, x_U, ncon, g_L, g_U, eval_jac_g.sparsity_indices,
                     eval_h.sparsity_indices, eval_f, eval_grad_f, eval_g, eval_jac_g)

#### Initialize the problem:

In [15]:
# initialize the problem:

x0 = np.array([1.0, 5.0, 5.0, 1.0])
pi0 = np.array([1.0, 1.0])
zl = np.zeros(nvar)
zu = np.zeros(nvar)
constraint_multipliers = np.zeros(ncon)

#### Invoke the solver:

In [ ]:
# invoking the solver:
_x, obj, status = nlp.solve(x0, mult_g=constraint_multipliers,
                            mult_x_L=zl, mult_x_U=zu)

# import pdb; pdb.set_trace()

#### Print the results:

In [ ]:
# print the solution:
def print_variable(variable_name, value):
    for i, val in enumerate(value):
        print("{}[{}] = {}".format(variable_name, i, val))


print("Solution of the primal variables, x")
print_variable("x", _x)

print("Solution of the bound multipliers, z_L and z_U")
print_variable("z_L", zl)
print_variable("z_U", zu)

print("Solution of the constraint multipliers, lambda")
print_variable("lambda", constraint_multipliers)

print("Objective value")
print("f(x*) = {}".format(obj))
